# Objective

Here, we'll run through our training data to gather information and try to achieve the best possible model

In [91]:
import numpy as np
from sklearn.linear_model import LogisticRegression
import csv
import pandas as pd
import time

In [118]:
# Functions

def addColumnThetaZero (array):
    return np.c_[np.ones(array.shape[0]), array]

def formatArray (dataFrame, columnToExtract) :
    array = dataFrame.values
    target = array[:,columnToExtract]
    params = np.delete(array, columnToExtract, axis = 1)
    return params, target

def loadFashionTrainData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_train.csv")

def loadFashionTestData():
    return pd.read_csv("fashion-mnist-dataset/fashion-mnist_test.csv")

def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

def regressionLogisticCostFunction (results, oposite, model, X):
    agaTheta = model.predict_proba(X)
    oneLessAgaTheta = np.ones((agaTheta.shape[0], agaTheta.shape[1]), dtype = float)
    oneLessAgaTheta = oneLessAgaTheta - agaTheta
    logAgaTheta = np.log10(agaTheta)
    logOneLessAgaTheta = np.log10(oneLessAgaTheta)
    test1 = logAgaTheta * results + logOneLessAgaTheta * oposite
    return (np.sum(test1)/(agaTheta.shape[0]))

def createTarget (target):
    results = np.zeros((target.size, 10), dtype=int)
    oposite = np.zeros((target.size, 10), dtype=int)
    for i in range(10):
        for j in range(target.size):
            if (target[j] != i):
                results[j][i - 1] = 0
                oposite[j][i - 1] = 1
            else:
                results[j][i - 1] = 1
                oposite[j][i - 1] = 0
    return results, oposite

48000


In [116]:
fashionTrainDataset = loadFashionTrainData()
fashionTestDataset = loadFashionTestData()

trainSet, validationSet = split_train_test(fashionTrainDataset, 0.2)

fashionTrainParams, fashionTrainTarget = formatArray(trainSet, 0)
fashionValidationSetParams, fashionValidationSetTarget = formatArray(validationSet, 0)

results, oposite = createTarget(fashionTrainTarget)

fashionTrainParams = addColumnThetaZero(fashionTrainParams)
fashionValidationSetParams = addColumnThetaZero(fashionValidationSetParams)
print (fashionTrainParams[:5])
print (type(fashionTrainParams))

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>


In [106]:
# C = 1.0
penalt = 'l1'
# stopFit = 0.0001
solverMode = 'newton-cg'
maxIter = 10
multiClass = 'multinomial'
# talk = True
# reuse = False

In [107]:
# logisticModel = LogisticRegression(C=C, penalty=penalt, tol = stopFit, solver = solverMode, max_iter = maxIter, multi_class = multiClass, verbose = talk, warm_start = reuse)
logisticModel = LogisticRegression(max_iter = maxIter, multi_class = multiClass, solver = solverMode)


In [108]:
start = time.clock()
logisticModel.fit(fashionTrainParams, fashionTrainTarget)
print (time.clock() - start)

print("Score : "+ str(logisticModel.score(fashionValidationSetParams, fashionValidationSetTarget)))

26.36764500000001
Score : 0.85175


/home/igoromote/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [109]:
import sklearn.metrics as metrics
print(metrics.accuracy_score(logisticModel.predict(fashionValidationSetParams), fashionValidationSetTarget))
print(metrics.precision_recall_fscore_support(logisticModel.predict(fashionValidationSetParams), fashionValidationSetTarget))

0.85175
(array([0.80844156, 0.95741056, 0.76583333, 0.8784219 , 0.77552743,
       0.90184049, 0.59292035, 0.94689542, 0.95213229, 0.95289256]), array([0.81639344, 0.96315338, 0.76328904, 0.84639255, 0.72476341,
       0.9440367 , 0.65921288, 0.91838352, 0.94391717, 0.9427637 ]), array([0.81239804, 0.96027339, 0.76455907, 0.86210984, 0.74928659,
       0.9224563 , 0.62431173, 0.93242156, 0.94800693, 0.94780107]), array([1220, 1167, 1204, 1289, 1268, 1090, 1118, 1262, 1159, 1223]))


In [119]:
costFunction = regressionLogisticCostFunction(results, oposite, logisticModel, fashionTrainParams)
print (costFunction)

-7.761845837062942
